In [1]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import copy
import cobra
import cobra.test
import math
from collections import defaultdict
#import efmtool
import matplotlib.pyplot as plt

In [2]:
import cvxpy as cp
import cvxopt
import cobra
import itertools
from cobra.flux_analysis import flux_variability_analysis

## Define functions:

In [3]:
def find_rxns_from_mets(model,met_list):
    import cobra
    import numpy as np

    if type(met_list) != list:
        met_list = [met_list]
    A = cobra.util.array.create_stoichiometric_matrix(model,'dense')
    rxns = [rxn.id for rxn in model.reactions]
    mets = [met.id for met in model.metabolites]
    if len(met_list) == 1:
        tmp_met = met_list[0]
        id_met = mets.index(tmp_met)
        outp = list(np.array(rxns)[np.where(A[id_met,:])])
    elif len(met_list) == 0:
        print('Put in a metabolite id!')
    else:
        # Find rxns that contain all metabolites:
        outp = []
        rxn_list = []
        for tmp_met in met_list:
            id_met = mets.index(tmp_met)
            rxn_list.append(list(np.array(rxns)[np.where(A[id_met,:])]))
        list_to_compare = rxn_list[0]
        for rxn_tmp in list_to_compare:
            count = 1
            for i in range(1,len(rxn_list)):
                if rxn_tmp in rxn_list[i]:
                    count += 1
            if count == len(rxn_list):
                outp.append(rxn_tmp)
    return outp

In [4]:
def convert_to_irreversible(cobra_model):
    """
    Split reversible reactions into two irreversible reactions

    These two reactions will proceed in opposite directions. This
    guarentees that all reactions in the model will only allow
    positive flux values, which is useful for some modeling problems.

    cobra_model:
        A cobra model object which will be modified in place.

    """
    from cobra import Reaction
    import pandas as pd
    from cobra.util import set_objective
    reactions_to_add = []
    coefficients = {}
    for reaction in cobra_model.reactions:
        # If a reaction is reverse only, the forward reaction (which
        # will be constrained to 0) will be left in the model.
        if reaction.lower_bound < 0:
            reverse_reaction = Reaction(reaction.id + "_b")
            reaction.id = reaction.id + "_f"
            reverse_reaction.lower_bound = max(0, -reaction.upper_bound)
            reverse_reaction.upper_bound = -reaction.lower_bound
            coefficients[reverse_reaction] = reaction.objective_coefficient * -1
            reaction.upper_bound = max(0, reaction.upper_bound)
            reaction.lower_bound = max(0, reaction.lower_bound)

            # Make the directions aware of each other
            reaction.notes["reflection"] = reverse_reaction.id
            reverse_reaction.notes["reflection"] = reaction.id
            reaction_dict = {k: v * -1 for k, v in pd.Series.iteritems(reaction._metabolites)}
            reverse_reaction.add_metabolites(reaction_dict)
            reverse_reaction._model = reaction._model
            reverse_reaction._genes = reaction._genes
            for gene in reaction._genes:
                gene._reaction.add(reverse_reaction)
            reverse_reaction.subsystem = reaction.subsystem
            #reverse_reaction._gene_reaction_rule = reaction._gene_reaction_rule
            #
            # ALO edited previous line as
            #
            reverse_reaction._gene_reaction_rule = reaction.gene_reaction_rule
            reactions_to_add.append(reverse_reaction)
    cobra_model.add_reactions(reactions_to_add)
    set_objective(cobra_model, coefficients, additive=True)
    return cobra_model

In [5]:
def MinNW(model, protected_mets, protected_rxns, functionality_df, selected_solver):
    '''
    A MILP formulation derived from (https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-016-1412-z) to
    reduce a Genome-Scale Metabolic Network so that EFM calculation is faster and macroreactions may be derived. This is 
    an approach that InSilico uses. I want to make sure that it works for my stuff as well. 
    This is the second MILP, MinNW, that finds the smallest network it can given a model and specific functionalities. 
    
    Inputs:
        model: A cobra genome-scale metabolic model
        protected_mets: A list of protected metabolites, i.e. metabolites that absolutely need to be in the resulting network
        
        protected_rxns: A list of protected reactions, i.e. reactions that absolutely need to be in the resulting network
        
        functionality_df: A (Fx3) pandas DataFrame containing the reactions along with their lower and upper bounds describing 
            the F functionalities desired in the resulting network. May be opposing ones (e.g. O2 uptake inhibited and allowed).
            The first column contains the name of the reactions of interest, the second contains the lower bound and third
            the upper bound for the reaction. May contain more than one reaction. Needs to contain an equivalent amount of bonds. 
        
    Outputs:
        prob.status: Status of solver, either 'infeasible' or 'optimal'
        
        model_dict: A dictionary containing the Stoichiometric matrix, reactions, metabolites and bounds for the new, reduced 
            network. 
    
    '''
    #import cvxpy as cp
    #import cvxopt
    #import cobra
    #import itertools
    #from cobra.flux_analysis import flux_variability_analysis
    
    #
    # ALO edited. Do not do imports inside a function, specially if you time it. imports should be at the top of a file always
    #
    
    A = cobra.util.array.create_stoichiometric_matrix(model,'dense')
    b = np.zeros(A.shape[0])
    rxns = [rxn.id for rxn in model.reactions]
    n_rxns = len(rxns)
    n_mets = A.shape[0]
    n_functions = functionality_df.shape[0]
    lb = [rxn.lower_bound for rxn in model.reactions]
    ub = [rxn.upper_bound for rxn in model.reactions]
    #M = max(ub)
    
    # Perform FVA to get correct M:
    #fva_df = flux_variability_analysis(model,fraction_of_optimum=0.99) # TODO: Get lowest objective value from functionality here..
    #new_ub = fva_df['maximum'].tolist()
    #M = np.array([new_ub]).T
    M = np.array([ub]).T
    #print(M)
    #print(M.shape)
    delta = 1e-6 # Between 1e-4 and 1e-6
    #delta = np.array([lb]).T # Between 1e-4 and 1e-6
    
    # Initiate an id_protected_rxns list:
    protected_rxns_orig = list(itertools.chain.from_iterable(protected_rxns))
    protected_met_rxns = []
    for met in protected_mets:
        protected_met_rxns_tmp = find_rxns_from_mets(model,met)
        if bool(set(protected_rxns_orig) & set(protected_met_rxns_tmp)):
            continue
        else:
            protected_met_rxns.append(protected_met_rxns_tmp)
    protected_met_rxns = list(set(list(itertools.chain.from_iterable(protected_met_rxns))))
    
    # Set constraints so that reactions associated with protected metabolites have at least 
    # an alpha of 1:
    id_met_rxn_protected = []
    for rxn in protected_met_rxns:
        id_met_rxn_protected.append(rxns.index(rxn))
    prot_met_vec = np.zeros(n_rxns)
    prot_met_vec[np.array(id_met_rxn_protected)] = 1
    
    id_rxn_protected = []
    for rxn in protected_rxns:
        id_rxn_protected.append(rxns.index(rxn))
    prot_rxn_vec = np.zeros(n_rxns)
    prot_rxn_vec[np.array(id_rxn_protected)] = 1
    
    # Add constraints to a matrix instead of multiple dictionaries:
    alpha_output = cp.Variable(n_rxns,boolean=True)
    alpha_orig = cp.Variable((n_rxns,n_functions),boolean=True) # Create a matrix of unknown alphas.
    v_vector_orig = cp.Variable((n_rxns,n_functions)) # Create a matrix of unknown fluxes.
    
    # Set upper and lower matrices:
    lb_mat = np.array(([lb]*n_functions)).T
    ub_mat = np.array(([ub]*n_functions)).T
    
    # Set bounds according to input df:
    for j in range(0,n_functions):
        # Get id of reactions in function first column:
        id_vec_tmp = [rxns.index(x) for x in functionality_df.iloc[j,0]]
        lb_mat[id_vec_tmp,j] = functionality_df.iloc[j,1]
        ub_mat[id_vec_tmp,j] = functionality_df.iloc[j,2]
    
    # Set the steady-state conditions:
    b_mat = np.zeros((n_functions,n_mets)).T
    
    # Set the inequalities:
    my_constraints = []
    my_constraints.append(A @ v_vector_orig == b_mat)
    my_constraints.append(v_vector_orig >= lb_mat)
    my_constraints.append(v_vector_orig <= ub_mat)
    my_constraints.append(delta*alpha_orig <= v_vector_orig)
    #my_constraints.append(0 <= v_vector_orig)
    my_constraints.append(v_vector_orig <= cp.multiply(alpha_orig,M))
    #my_constraints.append(v_vector_orig <= alpha_orig*M)
    
    # Add the protected reaction constraints:
    my_constraints.append(cp.sum(cp.multiply(prot_rxn_vec,alpha_output)) == len(id_rxn_protected))
    
    # Add the protected metabolite constraints:
    my_constraints.append(cp.sum(cp.multiply(prot_met_vec,alpha_output)) >= 1)
    
    # Calculate the sum of alphas:
    alpha_sum = cp.sum(alpha_orig, axis=1, keepdims=False)
    my_constraints.append(alpha_output <= alpha_sum)
    my_constraints.append(alpha_sum <= (1+n_functions)*alpha_output)
    
    # Solve!
    prob = cp.Problem(cp.Minimize(cp.sum(alpha_output)),constraints=my_constraints)
    prob.solve(solver = selected_solver) # TODO: CHANGE TO CPLEX!!!!
    
    # Output a new model (S-matrix):
    id_present_reactions = np.where(alpha_output.value)
    A2 = A.T[id_present_reactions].T
    id_metabolites_present = np.where(A2.any(axis=1))[0]
    A_new = np.delete(A2, np.where(~A2.any(axis=1))[0], 0)
    
    # Output new lbs, ubs, rxns and metabolites for the model:
    lb_new = list(np.array(lb)[id_present_reactions])
    ub_new = list(np.array(ub)[id_present_reactions])
    rxns_new = list(np.array(rxns)[id_present_reactions])
    metabolites = [met.id for met in model.metabolites]
    metabolites_new = list(np.array(metabolites)[id_metabolites_present])
    
    # Output a model as dictionary:
    model_dict = defaultdict()
    model_dict['A'] = A_new
    model_dict['rxns'] = rxns_new
    model_dict['mets'] = metabolites_new
    model_dict['ub'] = ub_new
    model_dict['lb'] = lb_new
    
    return prob.status,model_dict

## Load model:

In [6]:
model = cobra.test.create_test_model("textbook")
convert_to_irreversible(model)
A0 = cobra.util.array.create_stoichiometric_matrix(model,'dense')
print('E.coli model has dimensions: ',A0.shape)

E.coli model has dimensions:  (72, 141)


In [7]:
# Define the functionality for E.coli:
funcs = [['Biomass_Ecoli_core']]
lbs = [[0.99*0.87]]
ubs = [[1000]]
func_df_ecoli = pd.DataFrame([funcs,lbs,ubs]).T

## Perform the Network reduction for E.coli:

In [8]:
%%time
selected_solver = 'GLPK_MI'
stat_ecoli,model_dict_ecoli = MinNW(model, ['atp_c'], ['Biomass_Ecoli_core'], func_df_ecoli, selected_solver)
print(stat_ecoli)

# You could compare the "GLPK_MI" and the "CPLEX" solvers in terms of time. You could also load the larger
# E.coli model by using "ecoli" instead of "textbook" and see there how the performance of the algorithm is affected
# by model size .

optimal
CPU times: user 3.98 s, sys: 0 ns, total: 3.98 s
Wall time: 3.97 s


In [9]:
%%time
selected_solver = 'CPLEX'
stat_ecoli,model_dict_ecoli = MinNW(model, ['atp_c'], ['Biomass_Ecoli_core'], func_df_ecoli, selected_solver)
print(stat_ecoli)

optimal
CPU times: user 524 ms, sys: 14.2 ms, total: 538 ms
Wall time: 79.9 ms
